In [1]:
import random
from collections import MutableMapping
from random import randrange

/home/qiang/venv37/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


In [2]:
def random_seq(bases = "ATCG", k = 21):
    return "".join(random.choices(bases, k = k))

'''consider a DNA sequence with fixed length 21:
there are at most: pow(4, 21) = 4^21 = 4398046511104 possible sequences.
'''

def seq_hash_code(seq: str) -> int:
    """convert a dna sequence (as a key) to a hash code
    params
        seq: dna sequence with bases from: 'A','T','C','G'
    returns
        code: hash code for key k, int 
    """
    bases = "ATCG"
    codes = ["1"] # highest bit, if len(seq) is quarantee to fixed 21
                  # we may not use highest bit 1.
    for base in seq:
        # use two bits represent each base: A:00, T:01, C:10, G:11
        codes.append(bin(bases.index(base))[2:].zfill(2)) # binary code

    code = int("".join(codes), 2) # converto binary code to decimal
    return code



In [3]:
# MyProbHashMap class using MAD (multiply-add-and-divide, MAD) as compression 
# function and linear probing collision-handling scheme
class MyProbeHashMap(MutableMapping):
    class _Item:
        __slots__ = '_key', '_value'
        def __init__( self, k, v = None ):
            self._key = k
            self._value = v

        def __eq__( self, other ):
            return self._key == other._key

        def __ne__( self, other ):
            return not( self == other )

        def __lt__( self, other ):
            return self._key < other._key

        def __ge__( self, other ):
            return self._key >= other._key

        def __str__( self ):
            return "<" + str( self._key ) + "," + str( self._value ) + ">"

        def key( self ):
            return self._key

        def value( self ):
            return self._value    


    _AVAIL = object()
    _MIN_CAP = 11               # minimal capacity of the Map
    
    def __init__( self, cap = 11, p = 109345121, probing = 'linear' ):
        cap = max(MyProbeHashMap._MIN_CAP, cap)  # minimal capacity of Map
        # is this prime p big enough for k = 21?
        self._T = cap * [None]               # bucket array with cap entries
        self._n = 0                          # number of elements in array
        self._prime = p                      # prime number for MAD compression
        # a: scale
        self._scale = 1 + randrange( p - 1 ) # scale(also called: a) in [1, p-1]
        # seems we don't need this loop as p is prime
        trouve = False
        while not trouve:
            self._scale = 1 + randrange( p - 1 )
            if not ( self._scale % p ) == 0:
                trouve = True
        
        self._shift = randrange( p )         # shift(also called: b) in [0, p-1]
        self._mask = cap                     # N
        self._q = 7                          # used for double hash
        self._prob_func = self._probing_linear
        if probing == "quadratic":
            self._prob_func = self._probing_quadratic
        elif probing == "double_hash":
            self._prob_func = self._probing_double_hash
        elif probing == "random":
            self._prob_func = self._probing_random
            
        self._collisions = 0


    # hash function using _hash_code and MAD compression function
    def _hash_function( self, k ):
        tmp = seq_hash_code(k) * self._scale + self._shift 
        tmp = tmp  % self._prime % self._mask
        return tmp        
        
    def get_load_factor(self):
        return self._n / len(self._T)
        
    # length (number of elements) in the table(T)
    def __len__( self ):
        return self._n

    def _is_available( self, j ):
        return self._T[j] is None or self._T[j] is MyProbeHashMap._AVAIL
    

    # looking for an entry for a key k start from index j
    # we didn't use this function
    # Instead, we use _find_slot2 to support different probing methods
    # and to count collisions
    def _find_slot( self, j, k ):
        first_avail = None
        while True:
            if self._is_available( j ):
                if first_avail is None:
                    first_avail = j
                if self._T[j] is None:
                    return ( False, first_avail )

            elif k == self._T[j]._key:
                return ( True, j )
            # loop in the table
            j = (j + 1) % len( self._T )
            self._collisions += 1
            
    # three different probing methods to solve collision
    def _probing_linear(self, i, *args):
        return i
    
    def _probing_quadratic(self, i, *args):
        """probing quadratic"""
        return i*i
    
    def _probing_double_hash(self, i, k, *args):
        """double hashing
        f(i) = i * h'(k)
        h'(k) = q - (k mod q)
        """
        h_prime = self._q - (seq_hash_code(k) % self._q)
        return i * h_prime

    def _probing_random(self, i, k, *args):
        # this probing doesn't work well.
        random.seed(1)
        return i * random.randint(1, 7)
    
    def get_collisions(self):
        return self._collisions
    
    def reset_collisions(self):
        self._collisions = 0
        
    def _find_slot2(self, j, k):
        """compared with _find_slot, this method support three different
        probing methods, and we can count the collision numbers
        """
        first_avail = None
        i = 0
        while True:
            new_j = (j + self._prob_func(i, k)) % len(self._T)
            if self._is_available(new_j):# May be None or _AVAIL
                if first_avail is None:
                    first_avail = new_j
                if self._T[new_j] is None:
                    return (False, first_avail)
            elif k == self._T[new_j]._key:
                return (True, new_j)
            # collision:
            i += 1
            self._collisions += 1
                
            
    def _bucket_getitem( self, j, k ):
        found, s = self._find_slot2( j, k )
        if not found:
            return False

        return self._T[s]._value

    def _bucket_setitem( self, j, k, v ):
        found, s = self._find_slot2( j, k )
        if not found:
            self._T[s] = self._Item( k, v )
            self._n += 1
        else:
            self._T[s]._value = v

    def _bucket_delitem( self, j, k ):
        found, s = self._find_slot2( j, k )
        if not found:
            return False
        value = self._T[s]._value
        self._T[s] = MyProbeHashMap._AVAIL
        return value

    def keys(self):
        for key in self:
            yield key
            
    def values(self):
        for key in self:
            yield self[key]

    def __iter__( self ):
        for j in range( len( self._T ) ):
            if not self._is_available( j ):
                yield self._T[j]._key

    def __getitem__( self, k ):
        j = self._hash_function( k )
        tmp = self._bucket_getitem( j, k )
        if not tmp:
            raise KeyError
        else:
            return tmp

    def __setitem__( self, k, v ):
        j = self._hash_function( k )
        self._bucket_setitem( j, k, v )
        if self._n > len( self._T ) // 2:
            self._resize( 2 * len( self._T ) - 1 )

    def __delitem__( self, k ):
        j = self._hash_function( k )
        succes = self._bucket_delitem( j, k )
        if succes:
            self._n -= 1
            # need resize? minimal size = 
            if self._n < len(self._T) // 4:
                new_cap = max(self._min_cap, (len(self._T)+1)/2)
                self._resize(new_cap)
        else:
            raise KeyError

    def _resize( self, c ):
        old = list( self.items() )
        self._T = c * [None]
        self._n = 0
        self._mask = c
        for (k,v) in old:
            self[k] = v

    def is_empty( self ):
        return len( self ) == 0

    def __str__( self ):
        if self.is_empty():
            return "{}"
        pp = "{"
        for item in self.items():
            pp += str( item )
        pp += "}"
        pp += " size = "
        pp += str( len( self ) )
        return pp

    def get( self, k, d = None ):
        try:
            tmp = self[k]
            return tmp
        except KeyError:
            return d

    def setdefault( self, k, d = None ):
        try:
            tmp = self[k]
            return tmp
        except:
            self[k] = d
            return d

In [4]:
# test three probing methods and view their total collisions
# check all key and corresponding value are the same as they are stored into map
# chech there is no repeated keys in the map.
def test_probings():
    probe_methods = ['linear', 'quadratic', 'double_hash', 'random']
    #probe_methods = ['random']
    for probe in probe_methods:
        m = MyProbeHashMap(probing = probe)
        m.reset_collisions()
        n_keys_test = 10000
        for i in range(n_keys_test):
            seq21 = random_seq()
            m[seq21] = seq21

        print("{}/{} collisions/items occurs using probing {}".format(
            m.get_collisions(), n_keys_test, probe))

        for key, value in m.items():
            if not key == value:
                print("Error! key is not euqal to value")
                return False   

        keys = set(m.keys())
        if len(keys) != len(m):
            print("found repeated keys in map")

        print("test OK, map length:{}".format(len(m)))
              
test_probings()


14773/10000 collisions/items occurs using probing linear
test OK, map length:10000
12826/10000 collisions/items occurs using probing quadratic
test OK, map length:10000
12308/10000 collisions/items occurs using probing double_hash
test OK, map length:10000
9999/10000 collisions/items occurs using probing random
test OK, map length:2


In [5]:
def test_MyProbeHashMap():
    m = MyProbeHashMap()
    for i in range(100):
        seq = random_seq(k = 5)
        m[seq] = seq
    print("testing methods keys():")
    keys = []
    for key in m.keys():
        keys.append(key)
    print(len(keys) == len(m))

test_MyProbeHashMap()

testing methods keys():
True


In [8]:
class DeBrujinGraph:
    _bases = "ACTG"
    
    def __init__(self, nodes, k = 21):
        self._outgoing = MyProbeHashMap()
        self._incoming = MyProbeHashMap()
        self._directed = True       # indicates this is a directed graph
        self._k = k
        for node in nodes:
            print("nodes is:", node, type(node))
            # add insert vertex operation
            self._insert_vertex(node)
            for successor in self._all_possible_successors(node):
                if successor in self:
                    self._insert_edge(node, successor, successor[-1])
                    
    
    def __str__(self):
        s = "G( nodes{ "
        for v in self.vertices():
            s += str( v ) + " "
        s += "}, edge{ "
        for e in self.edges():
            s += str( e ) + " "
        s += "} )"
        return s

    def _all_possible_successors(self, N: str) -> [str]:
        return [N[1:]+base for base in DeBrujinGraph._bases]
        
    def _is_directed(self):
        return self._directed

    def _node_count(self):
        return len(self._outgoing)

    def _nodes(slef):
        return self._outgoing.keys()

    def _edge_count(self):
        total = sum(len(self._outgoing[v]) for v in self._outgoing)
        return total if self._is_directed() else total // 2
    
    def _edges(self):
        result = set()
        for secondary_map in self._outgoing.values():
            result.update(secondary_map.values())
        return result
    
    def _get_edge(self, u, v):
        self._validate(u)
        self._validate(v)
        secondary_map = self._outgoing.get(u)
        if secondary_map is None:
            raise KeyError("{} is not a start vertex".format(u))
        return secondary_map.get(v, None)
    
    def _degree(self, v, outgoing = True):
        self._validate(v)
        adj = self._outgoing if outgoing else self._incoming
        secondary_map = adj.get(v, None)
        if secondary_map is not None:
            return len(adj[v])
        return None
    
    def _incident_edges(self, v, outgoing = True):
        self._validate(v)
        adj = self._outgoing if outgoing else self._incoming
        secondary_map = adj.get(v, None)
        if secondary_map is None:
            return None
        for edge in adj[v].values():
            yield edge
            
    def _insert_vertex(self, x = None):
        print(type(x))
        self._validate(x)
        self._outgoing[x] = MyProbeHashMap()
        if self._is_directed():
            self._incoming = MyProbeHashMap()
        return x

    def _validate(x, edge = False):
        if x is None:
            raise ValueError("x is None")
        if not isinstance(x, str):
            raise TypeError("x should be an instance of str, now is:{}".format(type(x)))
        if is_edge and (len(x) != 1 or x not in DeBrujinGraph._bases):
            raise ValueError("x should be one of character in ACTG")
        if not is_edge and (len(x) != self._k):
            raise ValueError("x should have {} characters".format(self._k))
        
    def _insert_edge(self, u, v, x = None):
        self._validate(u)
        self._validate(v)
        self._validate(x, True)
        self._outgoing[u][v] = x
        self._incoming[v][u] = x

    def __contains__(self, N: str) -> bool:
        # détermine si le graphe de Brujin contient le noeud N
        self._validate(N)
        return not self._outgoing.get(N, None)

    def __iter__(self):
        return self.nodes() # retourne un itérable sur les noeuds du graphe
    
    def load_factor() -> float:
        # calcule le facteur de charge de la table de hachage sous-jacente
        return self._outgoing.get_load_factor()
        
    def add(self, N: str):
        # ajoute le noeud N au graphe
        
        self._insert_vertex(N)
    
    def remove(self, N: str):
        self._validate(N)
        del self._outgoing[N]
        if self._is_directed():
            del self._incoming[N]
        pass # enlève le noeud N du graphe
    
    def nodes(self):
        # retourne un itérable sur les noeuds du graphe
        for node in self._vertices():
            yield node
    
    def predecessors(self, N: str):
        # retourne tous les prédécesseur du noeud N
        self._validate(N)
        predecses = self._incoming.get(N, None)
        if predecses is None: 
            return None
        result = []
        for u in predecses:
            result.append(u)
        return result
        
    
    def successors(self, N: str):
        # retourne tous les successeurs du noeud N
        self._validate(N)    
        succeses = self._outgoing.get(N, None)
        if succeses is None:
            return None
        result = []    
        for v in succeses:
            result.append(v)
        return result

In [9]:
#test for graph 
seq = 'ACTGA'
k = 2
kmers = [seq[i:i+k] for i in range(len(seq) - k + 1)]

print(kmers)
print(seq)
graph = DeBrujinGraph(kmers, k=2)



print('************')
for node in graph.nodes():
    print(node, 'successor = ',graph.successors(node), "predecessors =",graph.predecessors(node) )


print('load factor =' , graph.load_factor())

##########################################
print('\r\n','### add GA ###')
graph.add("GA")
for i in range(len(graph.nodes())):
    kmer = graph.nodes()[i]
    print(' '*i + kmer, 'successor = ',graph.successors(kmer), "predecessors =",graph.predecessors(kmer) )

print('\r\n','### remove GA ###')
graph.remove("GA")
for i in range(len(graph.nodes())):
    kmer = graph.nodes()[i]
    print(' '*i + kmer, 'successor = ',graph.successors(kmer), "predecessors =",graph.predecessors(kmer) )

['AC', 'CT', 'TG', 'GA']
ACTGA
nodes is: AC <class 'str'>
<class 'str'>


TypeError: x should be an instance of str, now is:<class '__main__.DeBrujinGraph'>